### Symbolic treatment of the fluid equations

In this notebook we obtain the equations, and all relevant quantities starting from the generating function and in arbitrary dimensions. 
We spacialize for the conformal case, but this is not needed. 

We use a flat, constant metric, but it is not difficult to add an arbitrary metric latter on, even curvature.

In [193]:
using Symbolics
using LinearAlgebra
using Latexify
using Plots

In [321]:
include("symbolics_ext.jl")

Φ (generic function with 1 method)

The variables are going to be a vector-tensor $\zeta_A = (\zeta_a, \zeta_{ab})$ with the tensor part being symmetric and (latter) trace free. We are going to use both representations, either a big vector or a pair vector-tensor.
With functions going from one to the other.

We first define some tensors and take a symmetrize them using the upper triangular part as the relevant tensor part, contrary to take the average of the components

In [313]:
#D = Int64(4) #dimensions
D = Int64(3) #dimensions
L = (D+(D*(D+1)÷2))
ζ = Vector{Float64}(undef,L)
ζ_v = Vector{Float64}(undef,D)
ζ_t = Array{Float64}(undef,D,D)
g = make_g(D)

3×3 Matrix{Float64}:
 -1.0  0.0  0.0
  0.0  1.0  0.0
  0.0  0.0  1.0

In [276]:
v = [1;2;3;4;5;6]
sm = zeros(3,3)

vector2symmat!(v,sm)
sm

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 2.0  4.0  5.0
 3.0  5.0  6.0

In [277]:
symmat2vector(sm)

6-element Vector{Float64}:
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0

In [316]:
vector_unpack(rand(L))

([0.7799807495362714, 0.04580715901973642, 0.0028145169300755057], [0.6008546598955622 0.14354170003542155 0.8109571654286736; 0.14354170003542155 0.04708094995034284 0.2566104124783206; 0.8109571654286736 0.2566104124783206 0.5936211996546081])

In [279]:
v = [1;2;3;4;5;6;7;8;9;10;11;12;13;14]
vector_unpack(v)

([1, 2, 3, 4], [5.0 6.0 7.0 8.0; 6.0 9.0 10.0 11.0; 7.0 10.0 12.0 13.0; 8.0 11.0 13.0 14.0])

In [299]:
ζ_t = rand(D,D)
println(tr(ζ_t'*g'*g*ζ_t))



ζ_v = rand(D)
#ζ_v = [0.; 0.; 0.; 0.]

my_symmetrization!(ζ_t)

Tr = tr(g*ζ_t)
ζ_t = ζ_t - Tr*g/D

latexify(ζ_t)

tr(g*ζ_t)

3.1247489703116673


-5.551115123125783e-17

Likewise we define functions which take either a long vector or the pair vector-tensor

In [281]:
function Φ_vt(ζ_v, ζ_s, p)
    D = length(ζ_v)
    g = make_g(D)
    #println(g)
    χ₀ = p[1]
    χ₁ = p[2]
    χ₂ = p[3]
    #ζ_s = my_symmetrization!(Symbolics.scalarize(ζ_t))
    μ = Symbolics.scalarize(ζ_v)'*g*Symbolics.scalarize(ζ_v)
    ν = (g*Symbolics.scalarize(ζ_v))'*Symbolics.scalarize(ζ_s)*Symbolics.scalarize(g*ζ_v)
    l = Symbolics.scalarize(ζ_s)*Symbolics.scalarize(g*ζ_v)
    𝚶 = l'* g * l
    τ₂= tr(Symbolics.scalarize(ζ_s)*g'*Symbolics.scalarize(ζ_s)*g)
    D1 = D//2+1
    D2 = D//2+2
    return χ₀*μ^(1-D//2) + χ₁*ν*μ^(-D1) + χ₂*(τ₂ - 4*D1*𝚶*μ^(-1) + 2*D1*D2*ν^2*μ^(-2))*μ^(-D1) 
    # the function seems to be correct: tr(g*T)=0 and also A is fully symmetric after symmetrization 
    # on first two indices
    #return 𝚶
end

function Φ_v(ζ,p)
    ζ_v, ζ_t = vector_unpack(ζ)
    #ζ_t = vector2symmat(ζ[D+1:end])
    #return Φ_vt(ζ[1:D], ζ_t, p)
    return Φ_vt(ζ_v, ζ_t, p)
end

ζ_v,ζ_t = vector_unpack(v)
Φ_vt(ζ_v,ζ_t, [1.,1.,1.])#, ζ_t, [1.;1;1])
Φ_v(v, [1.,1.,1.])
#ζ_v
#ζ_t

0.6214398204617124

In [282]:
@variables zv[1:D], zs[1:D,1:D], pv[1:3]

Φ_vt(zv, zs, pv);

In [283]:
zvt = [zv[i] for i ∈ 1:D]
zst = vec([zs[i,j] for i ∈ 1:D, j ∈ 1:D])
JΦ_v = Symbolics.gradient(Φ_vt(zv, zs, pv), zvt);
JΦ_s = (reshape(Symbolics.gradient(Φ_vt(zv, zs, pv), zst), (D,D)) + reshape(Symbolics.gradient(Φ_vt(zv, zs, pv), zst), (D,D))')/2 ;

In [284]:
JΦ_v_exp = Symbolics.build_function(JΦ_v,zv, zs, pv);
JΦ_ve = eval(JΦ_v_exp[1]);
JΦ_ve(ζ_v,ζ_t, pv)

3-element Vector{Num}:
  0.024056261216234404pv[1] + 0.0573006777581139pv[2] + 0.44991507929335484pv[3]
 -0.04811252243246881pv[1] - 0.1707326316874414pv[2] - 2.967721964755377pv[3]
 -0.07216878364870322pv[1] - 0.31223022370237574pv[2] - 6.425922626349708pv[3]

In [285]:
JΦ_s_exp = Symbolics.build_function(JΦ_s, zv, zs, pv);
JΦ_se = eval(JΦ_s_exp[1]);
JΦ_se(ζ_v, ζ_t, pv);

In [286]:
T_s = Symbolics.jacobian(JΦ_ve(zv, zs, pv), zvt);

In [287]:
T_exp = Symbolics.build_function(T_s, zv, zs, pv);
T = eval(T_exp[1]);

In [301]:
tr(g*T(ζ_v, ζ_t, [1.0;1.0;1.0]))

1.4551915228366852e-11

In [302]:
tr(g*ζ_t)

-5.551115123125783e-17

In [303]:
A_s = Symbolics.jacobian(JΦ_ve(zv, zs, pv), zst);

In [304]:
A_exp = Symbolics.build_function(A_s, zv, zs, pv);
A = eval(A_exp[1]);

In [305]:
A_val = A(ζ_v, ζ_t, [1.0;1.0;1.0]);

In [306]:
AA = reshape(A_val', (D,D,D)); # last index is the vectorial one

In [307]:
AS = symm_A(AA)

3×3×3 Array{Float64, 3}:
[:, :, 1] =
 -10996.3    5562.75   10647.3
   5562.75  -2787.82   -5382.36
  10647.3   -5382.36  -10256.1

[:, :, 2] =
  5203.28  -2105.25   -5382.36
 -2105.25    759.743   2210.94
 -5382.36   2210.94    5521.95

[:, :, 3] =
  9980.76  -5382.36  -9573.57
 -5382.36   2877.46   5162.48
 -9573.57   5162.48   9102.85

In [308]:
tr_A(AS,g)

3-element Vector{Float64}:
 -2047.71194648665
  1078.4120312179612
  1999.5568241954807

In [309]:
tr_A(STF_A(AS,g),g)

3-element Vector{Float64}:
 0.0
 9.094947017729282e-13
 0.0

In [310]:
A_stf = STF_A(AS,g)

3×3×3 Array{Float64, 3}:
[:, :, 1] =
 -11678.8    5562.75  10647.3
   5562.75  -2105.25  -5382.36
  10647.3   -5382.36  -9573.57

[:, :, 2] =
  5562.75  -2105.25   -5382.36
 -2105.25    400.272   2210.94
 -5382.36   2210.94    5162.48

[:, :, 3] =
 10647.3   -5382.36  -9573.57
 -5382.36   2210.94   5162.48
 -9573.57   5162.48   8436.33

In [311]:
full_symmetry_check(A_stf)

0.0

In [312]:
A_s_dual = Symbolics.jacobian(JΦ_se(zv, zs, pv), zvt);

LoadError: MethodError: no method matching jacobian(::Matrix{Num}, ::Vector{Num})
[0mClosest candidates are:
[0m  jacobian([91m::AbstractVector{T} where T[39m, ::AbstractVector{T} where T; simplify) at /Users/reula/.julia/packages/Symbolics/ohRLi/src/diff.jl:368

We define now the conserved quantities and then the field-coordinate transformation.

In [190]:
function C(ζ,p,g,k::Int64) 
    D = length(Symbolics.scalarize(g)[:,1])
    ζ_v, ζ_t = Symbolics.scalarize(vector_unpack(ζ))
        AA = reshape(Symbolics.scalarize(A(ζ_v, ζ_t, p))', (D,D,D))[:,:,k]
        upper2symm!(AA)
        Tr = tr(AA)
        AA = AA - g*Tr/D
        return [T(ζ_v, ζ_t, p)[:,k]; symmat2vector(AA)]
end

C (generic function with 3 methods)

In [192]:
@variables zz[1:D+D*(D+1)÷2] gv[D,D]
zzt = [zz[i] for i ∈ 1:(D+D*(D+1)÷2)]
JC1_s = Symbolics.jacobian(C(zz,pv,gv,1),zzt)

LoadError: ArgumentError: Cannot convert Sym to Float64 since Sym is symbolic and Float64 is concrete. Use `substitute` to replace the symbolic unwraps.

In [179]:
C1 = C(rand(14),[1;1;2],g,1)

14-element Vector{Float64}:
       1.4130329360789362e8
      -1.6507657601303877e7
      -6.004861925844659e7
      -1.309977535294286e8
      -2.939008563162741e7
       2.8982488340497864e6
       5.644496230800373e6
       2.0631844600661937e7
       9.8829149787784e6
 -540786.6806530505
      -2.257338327981137e6
       7.871801780504785e6
      -8.599721307814535e6
      -8.601924443293815e6

In [180]:
T0, A0 = vector_unpack(C1)

([1.4130329360789362e8, -1.6507657601303877e7, -6.004861925844659e7, -1.309977535294286e8], [-2.939008563162741e7 2.8982488340497864e6 5.644496230800373e6 2.0631844600661937e7; 2.8982488340497864e6 9.8829149787784e6 -540786.6806530505 -2.257338327981137e6; 5.644496230800373e6 -540786.6806530505 7.871801780504785e6 -8.599721307814535e6; 2.0631844600661937e7 -2.257338327981137e6 -8.599721307814535e6 -8.601924443293815e6])

In [181]:
full_symmetry_check(A0)

0.0

In [182]:
function C2A!(ζ,c,p)
    χ, tol, iter_max, g, M, C, J = p
    while norm(C(ζ,χ,g,1) - c) > tol && iter < iter_max
            ζ = ζ - J(ζ,p,g) \ (C(ζ,χ,g,1) - c)
            iter = iter + 1
            if iter == iter_max 
            println("iter_max reached j = $j")
            end
    end
end

C2A! (generic function with 1 method)

In [172]:
p_N = [1.,1.,1.], 10^(-6), g, 1, C1, JC1

LoadError: UndefVarError: JC1 not defined

In [ ]:
function c_to_f!(flu, con, p)
    χ, tol, iter_max, N, M = p
    for j ∈ 1:M
        flu[1,j] = -flu[1,j]
        iter = 1
        while F(flu[:,j],con[:,j], χ)'*F(flu[:,j],con[:,j], χ) > tol && iter < iter_max
            flu[:,j] = NR_step!(F, Jac, flu[:,j], con[:,j], χ)
            iter = iter + 1
            if iter == iter_max 
            println("iter_max reached j = $j")
            end
        end
    #println(iter)
        flu[1,j] = -flu[1,j]
    end
    return flu[:,:]
end


Φ (generic function with 1 method)

In [361]:
include("symbolics_ext.jl")
@variables vs[1:L] ps[1:3] gs[D,D]
vst = [vs[i] for i ∈ 1:L]
pv = (ps, gs)
Jζ = Symbolics.jacobian(Φ(vs,pv),vst) 

LoadError: LoadError: MethodError: no method matching Matrix{Real}(::Int64, ::Int64)
[0mClosest candidates are:
[0m  Matrix{T}([91m::UniformScaling[39m, ::Integer, [91m::Integer[39m) where T at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/uniformscaling.jl:471
[0m  Array{T, N}([91m::Missing[39m, ::Any...) where {T, N} at baseext.jl:32
[0m  Array{T, N}([91m::Nothing[39m, ::Any...) where {T, N} at baseext.jl:31
[0m  ...
in expression starting at /Users/reula/Julia/Fluidos_Conformes/symbolics_ext.jl:178

In [362]:
ζ_v, ζ_t = vector_unpack(vs)
#ζ_v' * gs * ζ_v

#typeof(ζ_v)

LoadError: MethodError: no method matching Matrix{Real}(::Int64, ::Int64)
[0mClosest candidates are:
[0m  Matrix{T}([91m::UniformScaling[39m, ::Integer, [91m::Integer[39m) where T at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/uniformscaling.jl:471
[0m  Array{T, N}([91m::Missing[39m, ::Any...) where {T, N} at baseext.jl:32
[0m  Array{T, N}([91m::Nothing[39m, ::Any...) where {T, N} at baseext.jl:31
[0m  ...

In [359]:
Symbolics.scalarize(vs)[1:D]

3-element Vector{Num}:
 vs[1]
 vs[2]
 vs[3]

In [360]:
vector2symmat(Symbolics.scalarize(vs)[D+1:end])[:,:]

LoadError: ArgumentError: Cannot convert Sym to Float64 since Sym is symbolic and Float64 is concrete. Use `substitute` to replace the symbolic unwraps.

In [456]:
include("symbolics_ext.jl")
mu(rand(14))

-0.16746992454806928

In [457]:
get_dim(14)

4

In [458]:
v = zeros(14)
v[1] = 1
v[2] = 1
mu(v)

0.0

In [463]:
l_ind(3,2,4)

7

In [464]:
l_ind(3,2,4)

7

In [460]:
i = 6
j = 6
M = 6

(i-1)*M - (i-1)*(i)/2 + j 

21.0

In [461]:
i*(M-1) + j

36

In [421]:
7*4

28

In [ ]:
÷